In [1]:
import hdf5_getters as GETTERS
import h5py
import glob
from pyspark.sql import SparkSession
import pyspark
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

import numpy as np
import hdf5_getters as GETTERS
import h5py
import glob
import pandas as pd
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

#### 1. Crear la sesión con SPARK

In [11]:

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('XML ETL') \
    .master("local[*]") \
    .config('job.local.dir', 'file:/models/music-recommender-model') \
    .getOrCreate()

df = spark.range(10).toDF("number")
df.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
+------+



In [19]:
# Stop Spark session
spark.stop()

In [15]:
df_test = spark.range(10)  # 1 millón de filas
print(df_test.count())  # Debe devolver 1,000,000

10


#### 2 - Carga del Dataset

In [12]:
df = spark.read.parquet("/opt/bitnami/spark/datasets/millionsong.parquet")

In [13]:
df.show(10)

+------------------+--------------------+--------------------+-------+---+----+--------------+--------+------------+
|           song_id|               title|              artist|  tempo|key|mode|time_signature|loudness|danceability|
+------------------+--------------------+--------------------+-------+---+----+--------------+--------+------------+
|SOBLFFE12AF72AA5BA|              Scream|        Adelitas Way| 99.944|  1|   1|             4|  -4.769|         0.0|
|SOQPWCR12A6D4FB2A3|A Poor Recipe For...|   Western Addiction|125.475|  7|   1|             4|   -7.24|         0.0|
|SOMZWCG12A8C13C480|    I Didn't Mean To|              Casual| 92.198|  1|   0|             4| -11.197|         0.0|
|SOJDASC12A8C13EB49|The Lark In The C...|            Alquimia| 41.279|  2|   1|             4| -13.179|         0.0|
|SOCIWDW12A8C13D406|           Soul Deep|        The Box Tops|121.274|  6|   0|             4|  -9.843|         0.0|
|SOFRDWL12A58A7CEF7|        Hit Da Scene|    Quest_ Pup_ Kevo| 1

#### Entrenar el Modelo

In [14]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

feature_cols = [
    'tempo',
    'loudness',
    'danceability',
    'key',
    'mode',
    'time_signature'
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vec")
df_assembled = assembler.transform(df)

scaler = StandardScaler(inputCol="features_vec", outputCol="scaled_features")
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

kmeans = KMeans(
    featuresCol='scaled_features',
    predictionCol='cluster_id',
    k=20,
    seed=42
)

#Cálculo de centroides
model = kmeans.fit(df_scaled)

#Ver en que centroide cae cada canción
df_with_cluster = model.transform(df_scaled)

In [15]:
df_scaled.show(1)

+------------------+------+------------+------+---+----+--------------+--------+------------+--------------------+--------------------+
|           song_id| title|      artist| tempo|key|mode|time_signature|loudness|danceability|        features_vec|     scaled_features|
+------------------+------+------------+------+---+----+--------------+--------+------------+--------------------+--------------------+
|SOBLFFE12AF72AA5BA|Scream|Adelitas Way|99.944|  1|   1|             4|  -4.769|         0.0|[99.9440002441406...|[2.85098281878643...|
+------------------+------+------------+------+---+----+--------------+--------+------------+--------------------+--------------------+
only showing top 1 row



#### Recomendación 

In [7]:
import numpy as np
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeansModel

# Supongamos que tienes ya definido:
# model, scaler_model, assembler, df_scaled

# Simulamos la nueva canción
new_song = {
    'tempo': 120.0,
    'loudness': -5.0,
    'danceability': 1000,
    'key': 2.0,
    'mode': 1.0,
    'time_signature': 4.0,
    'song_id': 'NEW_SONG_ID'
}


# Convertimos a Spark DataFrame
new_song_df_pd = pd.DataFrame([new_song])
new_song_df = spark.createDataFrame(new_song_df_pd)

#  Ensamblamos y escalamos usando el pipeline original
new_song_vec = assembler.transform(new_song_df)
new_song_scaled = scaler_model.transform(new_song_vec)

# Hacemos la predicción de cluster
new_song_pred = model.transform(new_song_scaled)
predicted_cluster = new_song_pred.select(
    "cluster_id").collect()[0]["cluster_id"]

print(f"La nueva canción ha sido asignada al cluster: {predicted_cluster}")

# Buscamos canciones del dataset original en el mismo cluster

# Si no habías guardado los clusters en el df original:
df_with_cluster = model.transform(df_scaled)

# Filtramos por el mismo cluster
similar_songs = df_with_cluster.filter(
    df_with_cluster.cluster_id == predicted_cluster)

# Mostramos algunas recomendaciones (por ejemplo 3)
similar_songs.select("song_id").limit(3).show()

La nueva canción ha sido asignada al cluster: 5
+------------------+
|           song_id|
+------------------+
|SOBLFFE12AF72AA5BA|
|SONHOTT12A8C13493C|
|SOFSOCN12A8C143F5D|
+------------------+



In [14]:
metadata_df = pd.read_csv("songs_metadata.csv")

# Supongamos que estos son los song_ids recomendados que has obtenido de tu modelo
recommended_song_ids = [row['song_id']
                        for row in similar_songs.select("song_id").limit(3).collect()]


# Filtramos los metadatos para obtener nombre de la canción y artista
recommended_songs = metadata_df[metadata_df['song_id'].isin(
    recommended_song_ids)]

# Mostramos el resultado
print(recommended_songs[['song_id', 'title', 'artist']])

                song_id                              title           artist
0    SOZECOE12AB017E615  Diamonds Are A Girl's Best Friend  Gloria De Haven
84   SOPJEXC12A6D4FB529                Hot Dental Supplies      Marga Gomez
149  SOJXYFD12A8C143701         The Ferocious O' Flahertys       Joe Heaney


#### Guardar el modelo

In [15]:
from pyspark.ml import Pipeline

# Creamos el pipeline con los stages que usaste durante el entrenamiento
pipeline = Pipeline(stages=[assembler, scaler, kmeans])

# Lo ajustamos sobre el dataframe ya escalado
pipeline_model = pipeline.fit(df)

In [16]:
pipeline_model.write().overwrite().save("file:///models/music-recommender-model")

##### Guardar las canciones y a que cluster pertenecen

In [17]:
df_with_cluster.show(5)

+-------+--------+---------+----+----+--------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+--------------------+----------+
|  tempo|loudness| duration| key|mode|time_signature|           song_id|               title|              artist|     timbre_mean_0|     timbre_std_0|      timbre_mean_1|      timbre_std_1|      timbre_mean_2|      timbre_std_2|      timbre_mean_3|      timbre_std_3|     timbre_mean_4|      timbre_std_4|      timbre_mean_5|      timbre_std_5|      timbre_mean_6|     

In [18]:
df_export = df_with_cluster.select("title", "artist", "cluster_id")
df_export.write.csv("songs_with_cluster", header=True, mode="overwrite")

In [19]:
df_export.toPandas().to_csv("songs_with_cluster.csv", index=False)

#### Cargar el Modelo

In [5]:
from pyspark.ml import PipelineModel

loaded_model = PipelineModel.load("file:///models/music-recommender-model")

In [6]:
# Crear nueva canción con timbre simulado
new_song = {
    'tempo': 120.0,
    'loudness': -5.0,
    'danceability': 60,
    'key': 2.0,
    'mode': 1.0,
    'time_signature': 4.0,
    'track_name': 'New Song',
    'artist_name': 'Unknown Artist'
}



# Convertir a DataFrame de Spark
df_new = spark.createDataFrame(pd.DataFrame([new_song]))

# Predecir cluster directamente con el pipeline
predicted = loaded_model.transform(df_new)
predicted_cluster = predicted.select("cluster_id").collect()[0]["cluster_id"]

print(f"La canción fue asignada al cluster {predicted_cluster}")

df_with_clusters = loaded_model.transform(df_scaled)

recommendations = df_with_clusters.filter(
    df_with_clusters.cluster_id == predicted_cluster
).select("track_name", "artist_name").limit(3)

recommendations.show()


IllegalArgumentException: duration does not exist. Available: tempo, loudness, danceability, key, mode, time_signature, track_name, artist_name

In [15]:
metadata_df = pd.read_csv("songs_metadata.csv")

# Supongamos que estos son los song_ids recomendados que has obtenido de tu modelo
recommended_song_ids = [row['song_id']
                        for row in similar_songs.select("song_id").limit(3).collect()]


# Filtramos los metadatos para obtener nombre de la canción y artista
recommended_songs = metadata_df[metadata_df['song_id'].isin(
    recommended_song_ids)]

# Mostramos el resultado
print(recommended_songs[['song_id', 'title', 'artist']])

               song_id           title              artist
19  SOQBCSM12AC4687CDE  Pete's Crusade  Light Of The World


### Mejorar el modelo (Métricas)

In [8]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(
    featuresCol='scaled_features',
    predictionCol='cluster_id',
    metricName='silhouette',
    distanceMeasure='squaredEuclidean'
)

silhouette = evaluator.evaluate(df_with_cluster)
print(f"Silhouette score (k=20): {silhouette}")

Silhouette score (k=20): 0.33837674376824317


In [16]:
ks = list(range(2, 31))  # Por ejemplo, de 2 a 30 clusters

for k in ks:
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster_id', k=k, seed=42)
    model = kmeans.fit(df_scaled)
    df_clusters = model.transform(df_scaled)

    silhouette = evaluator.evaluate(df_clusters)
    print(f"k={k}, silhouette_score={silhouette:.4f}")

k=2, silhouette_score=0.3234


k=3, silhouette_score=0.4142


k=4, silhouette_score=0.4040


k=5, silhouette_score=0.3406


k=6, silhouette_score=0.3754


...

k=26, silhouette_score=0.3366


k=27, silhouette_score=0.3369


k=28, silhouette_score=0.3317


k=29, silhouette_score=0.3388


k=30, silhouette_score=0.3300


#### Que faeatures son más significativas

In [17]:
from pyspark.ml.feature import PCA

pca = PCA(k=6, inputCol="scaled_features", outputCol="pca_features")
model = pca.fit(df_scaled)
explained = model.explainedVariance.toArray()

for i, var in enumerate(explained):
    print(f"PC{i+1}: {var:.4f}")



PC1: 0.2542
PC2: 0.2323
PC3: 0.1915
PC4: 0.1630
PC5: 0.1590
PC6: 0.0000


25/07/02 15:23:09 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
